# Capstone Project - Best District for New Coffee Shop in Hong Kong

### Applied Data Science Capstone by IBM/Coursera

## 1. Introduction


### 1.1 Background

Hong Kong is primarily the entry into the international market because it scores high on several factors including its strategic location, productive work-force, attractive tax regime, world-class infrastructure and and an effective legal system. Many investors and entrepreneurs have chosen to set up their businesses in Hong Kong. Although Hong Kong is small in terms of the area of land, the population is comparatively large which contributes to the great business opportunities. 

### 1.2 Problem

Hong Kong is small, merely around 1,100 km², and the majority of Hong Kong's landscape consists of steep, undeveloped mountains and hills, which explain why Hong Kong has limited land for development. Officially, there are 18 districts in Hong Kong. The first question for a startup in Hong Kong would be where the shop will be. In the context of opening a new coffee shop, it is concerned that which district should be chosen regarding the business opportunities and competition. 

### 1.3 Stakeholders

The quantitative analysis aims to provide potential investors, or startup entrepreneurs, especially those who are interested in opening a new coffee shop with a guide to analyze the important problem scientifically. Supplement information such as rental prices of certain potential retail shops and their community facilities nearby is needed for more thorough consideration. Plus, government authorities can refer to the analysis for better understanding the city's culture diversity. 

## 2. Data


The analysis to find the best districts for new coffee shops is based on the following aspects:

* number of existing coffee shops in the districts;
* population density in the districts.

The sources of data are the following to achieve their respective aims:

* **Wikipedia**: To obtain the district data, including names of regions, names of districts, population density;
* **OpenCage Geocoder API**: To look up the latitudes and longitudes of all districts;
* **Foursquare API**: To obtain the number of coffee shop, their types and locations in all districts.

There are different websites scraping libraries and packages in Python. For scraping the table from Wiki, `pandas` is simply used to read the table into a pandas dataframe. Then, a free API, OpenCage Geocoder, is utilized to find the longitude-latitude coordinates for the list of districts in Hong Kong.

### 2.1 Downloading District Data from Wiki

Before scrapping and exploring the data, all the dependencies needed should be downloaded first.

In [1]:
import pandas as pd
!pip install lxml

!pip install opencage
from opencage.geocoder import OpenCageGeocode

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import requests

import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from pandas.io.json import json_normalize

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Next, `pandas` is used to read the tables in the Wikipedia page and a `for` loop is used to scrap a 'District table'. The name of a column is adjusted and the dataframe with the column names, Region, District, Population, Area(km²), and Density(/km²) is built. 

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/Districts_of_Hong_Kong', header=0)

headings = ['District']

for table in tables:
    current_headings = table.columns.values[:1]
    if len(current_headings) != len(headings):
        continue
    if all(current_headings == headings):
        break

df = table.rename(columns={"Population[when?] [6]":"Population",})
df = df[['Region','District','Population','Area(km²)','Density(/km²)']]

df

,Region,District,Population,Area(km²),Density(/km²)
0,Hong Kong Island,Central and Western,244600,12.44,19983.92
1,Hong Kong Island,Eastern,574500,18.56,31217.67
2,Hong Kong Island,Southern,269200,38.85,6962.68
3,Hong Kong Island,Wan Chai,150900,9.83,15300.10
4,Kowloon,Sham Shui Po,390600,9.35,41529.41
5,Kowloon,Kowloon City,405400,10.02,40194.70
6,Kowloon,Kwun Tong,641100,11.27,56779.05
7,Kowloon,Wong Tai Sin,426200,9.30,45645.16
8,Kowloon,Yau Tsim Mong,318100,6.99,44864.09
9,New Territories,Islands,146900,175.12,825.14


### 2.2 Generate Coordinate Data for All Districts

In order to utilize the Foursquare location data, it is needed to get the latitude and the longitude coordinates of each districts. OpenCage Geocoder, which is a free API that can be use to look up coordinates of places, and also find out the place a set of coordinates corresponds to, is used to get the data of latitudes and longitudes of the districts.

In [3]:
#Geocoding Tutorial from Amaral Lab: https://amaral.northwestern.edu/blog/getting-long-lat-list-cities

key = '1cfb1dbb86d54891a7c74a57c4761949'
geocoder = OpenCageGeocode(key)

In [4]:
list_lat = []
list_long = []

for index, row in df.iterrows():
    
    District = row['District']
    Region = row['Region']       
    query = str(District)+','+str(Region)
    
    geo_results = geocoder.geocode(query)   
    district_lat = geo_results[0]['geometry']['lat']
    district_long = geo_results[0]['geometry']['lng']
    
    list_lat.append(district_lat)
    list_long.append(district_long)

df['Latitude'] = list_lat
df['Longitude'] = list_long

df

,Region,District,Population,Area(km²),Density(/km²),Latitude,Longitude
0,Hong Kong Island,Central and Western,244600,12.44,19983.92,22.281938,114.158077
1,Hong Kong Island,Eastern,574500,18.56,31217.67,22.273078,114.233594
2,Hong Kong Island,Southern,269200,38.85,6962.68,22.244541,114.205376
3,Hong Kong Island,Wan Chai,150900,9.83,15300.10,22.279015,114.172483
4,Kowloon,Sham Shui Po,390600,9.35,41529.41,22.328190,114.160854
5,Kowloon,Kowloon City,405400,10.02,40194.70,22.330160,114.189937
6,Kowloon,Kwun Tong,641100,11.27,56779.05,22.312937,114.225610
7,Kowloon,Wong Tai Sin,426200,9.30,45645.16,22.341654,114.193859
8,Kowloon,Yau Tsim Mong,318100,6.99,44864.09,22.302857,114.182032
9,New Territories,Islands,146900,175.12,825.14,22.230076,113.986785


In [5]:
print('The dataframe has {} regions and {} districts.'.format(
        len(df['Region'].unique()),
        df.shape[0]
    )
)

The dataframe has 3 regions and 18 districts.


## 3. Methodology


The dataset will be first explored through looking at Hong Kong as a whole area, searching for venues in the area with their categories. With the help of geopy library and Foursquare API respectively, a Hong Kong district map and a list of venues in the first explored district, Central and Western, will be created. The purpose of doing so is to test the feasibility of the code and the accuracy of the Foursquare API.

After that, the same process will be done to all the districts in Hong Kong so as to generate a dataframe consisting of venues in each district. One hot encoding and GroupBy in Python will then be used to help create a new dataframe displaying the top 10 venues for each district.

Finally, k-Means Clustering is adopted to cluster the districts by looking at the similarities of the categories of the top venues in those areas. 

To achieve the aim of the study, coffee-shop-dominating clusters will be avoided. By choosing the clusters or districts with coffee shop being the least common venue, the desired outcome will be ideally be produced. However, there may be more than one potential districts or clusters, so the population should then be considered to choose the best district to open a coffee shop. 

## 4. Analysis

### 4.1 Exploring Dataset

#### Use geopy library to get the latitude and longitude values of Hong Kong

In order to define an instance of the geocoder, it is needed to define a user_agent. We will name our agent <em>hk_explorer</em>, as shown below.

In [6]:
address = 'Hong Kong'

geolocator = Nominatim(user_agent="hk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hong Kong are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hong Kong are 22.2793278, 114.1628131.


#### Create a Map of Hong Kong with Districts Superimposed on Top

In [7]:
# create map of Hong Kong using latitude and longitude values
map_hongkong = folium.Map(location=[latitude+0.08, longitude],zoom_start=11)

# add markers to map
for lat, lng, region, district in zip(df['Latitude'], df['Longitude'], df['Region'], df['District']):
    label = '{}, {}'.format(district, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hongkong)  

map_hongkong

In [8]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.ibb.co/bgWWs7w/Capture1.png")

#### Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = '1FZMJSXAV4VP3X0THT2EI1SF0EA5YH05T3JKYV0YDS5BAOVJ' # your Foursquare ID
CLIENT_SECRET = 'GD3V0SHKWYOQKA0XR1WF3CCOOIRXMTIIQ3LSA4ASYSCJ5WTR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1FZMJSXAV4VP3X0THT2EI1SF0EA5YH05T3JKYV0YDS5BAOVJ
CLIENT_SECRET:GD3V0SHKWYOQKA0XR1WF3CCOOIRXMTIIQ3LSA4ASYSCJ5WTR


#### Exploring First District in the Dataframe

Getting the district's name:

In [10]:
df.loc[0, 'District']

'Central and Western'

Geting the district's latitude and longitude values:

In [11]:
district_latitude = df.loc[0, 'Latitude']
district_longitude = df.loc[0, 'Longitude']

district_name = df.loc[0, 'District']

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Central and Western are 22.2819378, 114.1580765.


#### Getting Top 100 Venues in the First District within Radius of 500 Meters

First, the GET request URL should be created. The URL is named **url**.

In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=1FZMJSXAV4VP3X0THT2EI1SF0EA5YH05T3JKYV0YDS5BAOVJ&client_secret=GD3V0SHKWYOQKA0XR1WF3CCOOIRXMTIIQ3LSA4ASYSCJ5WTR&v=20180605&ll=22.2819378,114.1580765&radius=500&limit=100'

The GET request is sent and the resutls are examined.

In [13]:
url_results = requests.get(url).json()
url_results

{'meta': {'code': 200, 'requestId': '5e3bcb120de0d9001becc717'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Central and Western District',
  'headerFullLocation': 'Central and Western District, Hong Kong',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 176,
  'suggestedBounds': {'ne': {'lat': 22.286437804500007,
    'lng': 114.1629305584479},
   'sw': {'lat': 22.277437795499996, 'lng': 114.15322244155212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5305caeb498e391fd4b9caf0',
       'name': 'Mott 32 (卅二公館)',
       'location': {'address': 'B/F, Standard Chartered Bank Building, 4-4A Des Voeux Rd C',
        'lat': 22.280696467137126,
        'lng': 114.1

As all the information is in the *items* key, the **get_category_type** function should be borrowed from the Foursquare lab before proceeding.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

It is now ready to clean the json and structure it into a *pandas* dataframe.

In [15]:
venues = url_results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Mott 32 (卅二公館),Dim Sum Restaurant,22.280696,114.159380
1,Mandarin Oriental Hong Kong (香港文華東方酒店),Hotel,22.281857,114.159382
2,Mandarin Grill + Bar (文華扒房＋酒吧),Steakhouse,22.281928,114.159408
3,The Mandarin Cake Shop,Bakery,22.281959,114.159416
4,XYZ,Cycle Studio,22.280877,114.157108


Finally, the number of venues returned by Foursquare:

In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### 4.2 Exploring Districts in Hong Kong

#### Create a Function to Repeat the Same Process to All the Districts in Hong Kong

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        url_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in url_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the Function Above on Each District & Create a New Dataframe Called *district_venues*

In [18]:
district_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Central and Western
Eastern
Southern
Wan Chai
Sham Shui Po
Kowloon City
Kwun Tong
Wong Tai Sin
Yau Tsim Mong
Islands
Kwai Tsing
North
Sai Kung
Sha Tin
Tai Po
Tsuen Wan
Tuen Mun
Yuen Long


#### Check the Properties of the Resulting Dataframe

Dataframe Size:

In [19]:
print(district_venues.shape)
district_venues.head()

(755, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central and Western,22.281938,114.158077,Mott 32 (卅二公館),22.280696,114.159380,Dim Sum Restaurant
1,Central and Western,22.281938,114.158077,Mandarin Oriental Hong Kong (香港文華東方酒店),22.281857,114.159382,Hotel
2,Central and Western,22.281938,114.158077,Mandarin Grill + Bar (文華扒房＋酒吧),22.281928,114.159408,Steakhouse
3,Central and Western,22.281938,114.158077,The Mandarin Cake Shop,22.281959,114.159416,Bakery
4,Central and Western,22.281938,114.158077,XYZ,22.280877,114.157108,Cycle Studio


Numbers of Venues Returned for Each District:

In [20]:
district_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Central and Western,100,100,100,100,100,100
Eastern,3,3,3,3,3,3
Islands,2,2,2,2,2,2
Kowloon City,69,69,69,69,69,69
Kwun Tong,64,64,64,64,64,64
North,100,100,100,100,100,100
Sai Kung,54,54,54,54,54,54
Sha Tin,56,56,56,56,56,56
Sham Shui Po,33,33,33,33,33,33


Numbers of Unique Categories that can be Curated from All the Returned Venues:

In [21]:
print('There are {} uniques categories.'.format(len(district_venues['Venue Category'].unique())))

There are 160 uniques categories.


### 4.3 Examining Each District

In [22]:
# one hot encoding
district_onehot = pd.get_dummies(district_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
district_onehot['District'] = district_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [district_onehot.columns[-1]] + list(district_onehot.columns[:-1])
district_onehot = district_onehot[fixed_columns]

district_onehot.head()

,District,Adult Boutique,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Australian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Yunnan Restaurant,Zoo
0,Central and Western,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Central and Western,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Central and Western,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central and Western,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central and Western,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [23]:
district_onehot.shape

(755, 161)

#### Group Rows by District and by Taking the Mean of the Frequency of Occurrence of Each Category

In [24]:
district_grouped = district_onehot.groupby('District').mean().reset_index()
district_grouped

,District,Adult Boutique,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Australian Restaurant,...,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Yunnan Restaurant,Zoo
0,Central and Western,0.00,0.01,0.000000,0.01,0.000000,0.01,0.010000,0.000000,0.00,...,0.02,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.03,0.000000,0.00
1,Eastern,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00
2,Islands,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00
3,Kowloon City,0.00,0.00,0.000000,0.00,0.000000,0.00,0.028986,0.000000,0.00,...,0.00,0.00,0.000000,0.014493,0.00,0.00,0.000000,0.00,0.000000,0.00
4,Kwun Tong,0.00,0.00,0.000000,0.00,0.015625,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.000000,0.015625,0.00,0.00,0.000000,0.00,0.000000,0.00
5,North,0.01,0.00,0.020000,0.01,0.030000,0.00,0.020000,0.000000,0.01,...,0.00,0.01,0.000000,0.010000,0.01,0.01,0.020000,0.01,0.000000,0.00
6,Sai Kung,0.00,0.00,0.018519,0.00,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.018519,0.018519,0.00,0.00,0.018519,0.00,0.000000,0.00
7,Sha Tin,0.00,0.00,0.000000,0.00,0.000000,0.00,0.035714,0.000000,0.00,...,0.00,0.00,0.000000,0.017857,0.00,0.00,0.000000,0.00,0.000000,0.00
8,Sham Shui Po,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.000000,0.030303,0.00,0.00,0.000000,0.00,0.000000,0.00
9,Southern,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00


#### Confirm the New Size

In [25]:
district_grouped.shape

(17, 161)

#### Print Each District along with the Top 5 Most Common Venues

In [26]:
num_top_venues = 5

for hood in district_grouped['District']:
    print("----"+hood+"----")
    temp = district_grouped[district_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central and Western----
                 venue  freq
0  Japanese Restaurant  0.05
1   Italian Restaurant  0.05
2          Coffee Shop  0.04
3   Chinese Restaurant  0.04
4               Lounge  0.04


----Eastern----
       venue  freq
0   Bus Stop  0.67
1     Castle  0.33
2   Pet Café  0.00
3  Pet Store  0.00
4   Pharmacy  0.00


----Islands----
                   venue  freq
0     Rock Climbing Spot   0.5
1               Mountain   0.5
2         Adult Boutique   0.0
3  Portuguese Restaurant   0.0
4              Pet Store   0.0


----Kowloon City----
                venue  freq
0     Thai Restaurant  0.19
1        Dessert Shop  0.14
2                Café  0.07
3         Coffee Shop  0.06
4  Chinese Restaurant  0.06


----Kwun Tong----
                  venue  freq
0    Chinese Restaurant  0.11
1                  Café  0.08
2           Coffee Shop  0.08
3  Fast Food Restaurant  0.06
4   Japanese Restaurant  0.05


----North----
                venue  freq
0  Italian Restaurant  0.07

#### Put the Result into a *pandas* Dataframe wiht Venues in Descending Order

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the New Dataframe & Display the Top 10 Venues for Each District

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = district_grouped['District']

for ind in np.arange(district_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(district_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central and Western,Italian Restaurant,Japanese Restaurant,Lounge,Coffee Shop,Chinese Restaurant,Social Club,Bar,Cocktail Bar,Gym,Cantonese Restaurant
1,Eastern,Bus Stop,Castle,Zoo,Gym,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint
2,Islands,Mountain,Rock Climbing Spot,Zoo,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant
3,Kowloon City,Thai Restaurant,Dessert Shop,Café,Coffee Shop,Chinese Restaurant,Cha Chaan Teng,Hotpot Restaurant,Noodle House,Halal Restaurant,Bakery
4,Kwun Tong,Chinese Restaurant,Coffee Shop,Café,Fast Food Restaurant,Japanese Restaurant,Restaurant,Sushi Restaurant,Cantonese Restaurant,Cha Chaan Teng,Hong Kong Restaurant
5,North,Italian Restaurant,French Restaurant,Coffee Shop,Cocktail Bar,Mexican Restaurant,Ice Cream Shop,Rock Club,Pizza Place,Hotel,Art Gallery
6,Sai Kung,Seafood Restaurant,Café,Thai Restaurant,Dessert Shop,Coffee Shop,Chinese Restaurant,Pizza Place,Pub,Burger Joint,Restaurant
7,Sha Tin,Café,Shopping Mall,Coffee Shop,Dessert Shop,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Asian Restaurant
8,Sham Shui Po,Noodle House,Chinese Restaurant,Hong Kong Restaurant,Dessert Shop,Snack Place,Shopping Mall,Café,Italian Restaurant,Electronics Store,Vietnamese Restaurant
9,Southern,Reservoir,Zoo,Food Court,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant


### 4.4 Clustering  Districts

#### Run *k*-means to cluster the districts into 6 clusters.

In [29]:
# set number of clusters
kclusters = 6

district_grouped_clustering = district_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(district_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 2, 5, 5, 5, 5, 5, 0, 1], dtype=int32)

#### Create a New Dataframe for the Cluster and the Top 10 Venues for Each District

In [30]:
# add clustering labels
district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

district_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
district_merged = district_merged.join(district_venues_sorted.set_index('District'), on='District', how='right')

district_merged.head() # check the last columns!

,Region,District,Population,Area(km²),Density(/km²),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hong Kong Island,Central and Western,244600,12.44,19983.92,22.281938,114.158077,5,Italian Restaurant,Japanese Restaurant,Lounge,Coffee Shop,Chinese Restaurant,Social Club,Bar,Cocktail Bar,Gym,Cantonese Restaurant
1,Hong Kong Island,Eastern,574500,18.56,31217.67,22.273078,114.233594,3,Bus Stop,Castle,Zoo,Gym,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint
2,Hong Kong Island,Southern,269200,38.85,6962.68,22.244541,114.205376,1,Reservoir,Zoo,Food Court,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant
3,Hong Kong Island,Wan Chai,150900,9.83,15300.10,22.279015,114.172483,5,Café,Coffee Shop,Hotel,Italian Restaurant,Lounge,Hong Kong Restaurant,Japanese Restaurant,Korean Restaurant,Cantonese Restaurant,Chinese Restaurant
4,Kowloon,Sham Shui Po,390600,9.35,41529.41,22.328190,114.160854,0,Noodle House,Chinese Restaurant,Hong Kong Restaurant,Dessert Shop,Snack Place,Shopping Mall,Café,Italian Restaurant,Electronics Store,Vietnamese Restaurant


#### Visualize the Resulting Clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude+0.08, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(district_merged['Latitude'], district_merged['Longitude'], district_merged['District'], district_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.ibb.co/W2ngPx3/Capture2.png")

### 4.5 Examining Clusters

Each cluster is examined and the discriminating venue categories that distinguish each cluster are determined. Based on the defining categories, a name to each cluster will be assgined. 

#### Cluster 1 - Chinese Restaurants Dominating Areas (1)

In [33]:
district_merged.loc[district_merged['Cluster Labels'] == 0, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Sham Shui Po,390600,22.328190,114.160854,0,Noodle House,Chinese Restaurant,Hong Kong Restaurant,Dessert Shop,Snack Place,Shopping Mall,Café,Italian Restaurant,Electronics Store,Vietnamese Restaurant
14,Tai Po,307100,22.449402,114.171133,0,Chinese Restaurant,Fast Food Restaurant,Shopping Mall,Noodle House,Sushi Restaurant,Department Store,Café,Pub,Market,Coffee Shop
17,Yuen Long,607200,22.442646,114.030434,0,Chinese Restaurant,Noodle House,Dessert Shop,Fast Food Restaurant,Shopping Mall,Japanese Restaurant,Ramen Restaurant,Market,Burger Joint,Café


#### Cluster 2 - Undeveloped Area (1)

In [34]:
district_merged.loc[district_merged['Cluster Labels'] == 1, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Southern,269200,22.244541,114.205376,1,Reservoir,Zoo,Food Court,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant


#### Cluster 3 - Undeveloped Area (2)

In [35]:
district_merged.loc[district_merged['Cluster Labels'] == 2, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Islands,146900,22.230076,113.986785,2,Mountain,Rock Climbing Spot,Zoo,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint,French Restaurant


#### Cluster 4 - Populated Developed Area (1)

In [36]:
district_merged.loc[district_merged['Cluster Labels'] == 3, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Eastern,574500,22.273078,114.233594,3,Bus Stop,Castle,Zoo,Gym,Greek Restaurant,Gourmet Shop,Gay Bar,Gastropub,Furniture / Home Store,Fried Chicken Joint


#### Cluster 5 - Café / Coffee Shops Dominating Area (1)

In [37]:
district_merged.loc[district_merged['Cluster Labels'] == 4, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Wong Tai Sin,426200,22.341654,114.193859,4,Fast Food Restaurant,Coffee Shop,Food Court,Astrologer,Park,Szechuan Restaurant,Temple,Chinese Restaurant,Burger Joint,Pool
15,Tsuen Wan,303600,22.371661,114.113470,4,Chinese Restaurant,Shopping Mall,Noodle House,Fast Food Restaurant,Dessert Shop,Coffee Shop,Cha Chaan Teng,Hong Kong Restaurant,Ramen Restaurant,Department Store
16,Tuen Mun,495900,22.390826,113.973169,4,Coffee Shop,Shopping Mall,Cantonese Restaurant,Zoo,Hong Kong Restaurant,Department Store,Electronics Store,Fast Food Restaurant,Chinese Restaurant,Park


#### Cluster 6 - Café / Coffee Shops Dominating Area (2)

In [38]:
district_merged.loc[district_merged['Cluster Labels'] == 5, district_merged.columns[[1] + [2] + list(range(5, district_merged.shape[1]))]]

,District,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central and Western,244600,22.281938,114.158077,5,Italian Restaurant,Japanese Restaurant,Lounge,Coffee Shop,Chinese Restaurant,Social Club,Bar,Cocktail Bar,Gym,Cantonese Restaurant
3,Wan Chai,150900,22.279015,114.172483,5,Café,Coffee Shop,Hotel,Italian Restaurant,Lounge,Hong Kong Restaurant,Japanese Restaurant,Korean Restaurant,Cantonese Restaurant,Chinese Restaurant
5,Kowloon City,405400,22.330160,114.189937,5,Thai Restaurant,Dessert Shop,Café,Coffee Shop,Chinese Restaurant,Cha Chaan Teng,Hotpot Restaurant,Noodle House,Halal Restaurant,Bakery
6,Kwun Tong,641100,22.312937,114.225610,5,Chinese Restaurant,Coffee Shop,Café,Fast Food Restaurant,Japanese Restaurant,Restaurant,Sushi Restaurant,Cantonese Restaurant,Cha Chaan Teng,Hong Kong Restaurant
8,Yau Tsim Mong,318100,22.302857,114.182032,5,Hotel,Coffee Shop,Buffet,Burger Joint,Chinese Restaurant,Hong Kong Restaurant,Stadium,Train Station,Cocktail Bar,Café
11,North,310800,40.722105,-73.988081,5,Italian Restaurant,French Restaurant,Coffee Shop,Cocktail Bar,Mexican Restaurant,Ice Cream Shop,Rock Club,Pizza Place,Hotel,Art Gallery
12,Sai Kung,448600,22.382249,114.272828,5,Seafood Restaurant,Café,Thai Restaurant,Dessert Shop,Coffee Shop,Chinese Restaurant,Pizza Place,Pub,Burger Joint,Restaurant
13,Sha Tin,648200,22.381056,114.188879,5,Café,Shopping Mall,Coffee Shop,Dessert Shop,Fast Food Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Asian Restaurant


## 5. Results

The resulting clusters are as follow :

* Cluster 1 - Chinese Restaurants Dominating Areas
* Cluster 2 - Undeveloped Area (1)
* Cluster 3 - Undeveloped Area (2)
* Cluster 4 - Populated Developed Area (1)
* Cluster 5 - Café / Coffee Shops Dominating Area (1)
* Cluster 6 - Café / Coffee Shops Dominating Area (2)

From the result, the potential clusters are Clusters 1 to 4. However, considering the population, Clusters 2 and 3 are far less populated, meaning that the number of potential customers is less and so the business opportunity is less desirable. Besides, from the top 10 common venues, there is no café or coffee shop in Cluster 4. On the other hand, there are cafés in top 7 to 10 common venues in districts in Cluster 1. **Therefore, it is better to choose Cluster 4 which consists of just one district, Eastern, as the best district for opening a coffee shop.**

## 6. Discussion

It is observed that the results of the study heavily rely on the venues generated from the Foursquare API. It is suggested to repeat the study with other API so as to avoid inaccurate results due to incomplete or not up-to-date venue library.


## 7. Conclusion

From the analysis and results above, it is concluded to choose the Eastern district in Hong Kong to open a new coffee shop.